# Model Training

### Imports

In [2]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import boto3
# import re
# import os
# import io
# import time
# import tarfile
# import random
# import string
# import matplotlib.pyplot as plt
# from io import BytesIO
# from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.cluster import KMeans
# import sagemaker
# from sagemaker import PCA, KMeans, get_execution_role, Session
# from sagemaker import KMeansModel, KMeansModel
# from sklearn.metrics import silhouette_score
# from sagemaker.model import Model
# from sagemaker.predictor import Predictor
# import json
# from sagemaker.serializers import JSONSerializer
# from sagemaker.serializers import CSVSerializer
# from sagemaker.deserializers import JSONDeserializer
# import pickle

## Load Training Dataset from S3

In [3]:
# train_df = pd.read_csv('s3://team6datasets/splitDatasets/train.csv')

In [4]:
# train_df.shape

In [5]:
# train_df.head()

## Sagemaker Session and Role

In [6]:
# # Define the SageMaker session and role
# sagemaker_session = Session()
# role = get_execution_role()

## Dimensionality Reduction using Variance

In [7]:
# # Set the variance threshold
# variance_threshold = 0.2

# # Create a VarianceThreshold object
# selector = VarianceThreshold(threshold=variance_threshold)

# # Fit the selector to the data 
# selector.fit(train_df)

# # Get the support mask 
# support = selector.get_support(indices=True)

# # Select features based on the support mask
# reduced_features = train_df.iloc[:, support]  

# # Now 'reduced_features' is a DataFrame containing features with variance exceeding the threshold
# print(f"Number of features removed: {len(train_df.columns) - len(reduced_features.columns)}")

In [8]:
# # Show the reduced_features DataFrame
# print("Reduced Features DataFrame:")
# reduced_features.head()

In [9]:
# reduced_features.shape

## Model Training with KMeans Algorithm

### Use Elbow Method to Determine Ideal K

In [10]:
# # Check if the image file already exists
# if os.path.isfile('elbow_method.png'):
#     print("Elbow method already run. Image found in folder.")
# else:
#     # Define a range of k values
#     k_values = range(2, 11)  

#     # Convert the reduced_features DataFrame to float32
#     reduced_features_float32 = reduced_features.astype('float32')

#     # Train KMeans models for each k value and compute WCSS
#     wcss_values = []
#     for k in k_values:
#         kmeans = KMeans(role=role,
#                         instance_count=1,
#                         instance_type='ml.c4.xlarge',
#                         k=k)
#         kmeans.fit(kmeans.record_set(reduced_features_float32.values)) 
#         # Filter the DataFrame for the row where metric_name is 'train:msd'
#         wcss_row = kmeans.training_job_analytics.dataframe().loc[kmeans.training_job_analytics.dataframe()['metric_name'] == 'train:msd']
#         # Extract the WCSS value from the 'value' column of the filtered row
#         wcss_value = wcss_row['value'].iloc[0]
#         # Append the WCSS value to the list
#         wcss_values.append(wcss_value)

#     # Plot the WCSS values against k
#     plt.plot(k_values, wcss_values, marker='o')
#     plt.title('Elbow Method')
#     plt.xlabel('Number of clusters (k)')
#     plt.ylabel('Within-cluster sum of squares (WCSS)')
#     plt.savefig('elbow_method.png')  # Save the plot as an image
#     plt.show()
#     print("Elbow method run successfully. Image saved in folder.")


In [11]:
# # Define a function to generate a random 3-digit serial number
# def generate_serial_number():
#     return ''.join(random.choices(string.digits, k=3))

# # Drop the CustomerID column from the feature DataFrame
# features_for_training = reduced_features.drop(columns=['CustomerID'])

# # Store CustomerID separately for reference
# customer_ids = reduced_features['CustomerID']

# # Initialize S3 client
# s3 = boto3.client('s3')

# # Define hyperparameters for KMeans model
# hyperparameters = {'k': 4, 'init_method': 'random'}

# # Set the output path
# output_path = 's3://team6datasets/kmeans_model_output/'

# # Set k value
# k_value = hyperparameters['k']
# model_output_path = f'{output_path}kmeans-{k_value}/output/'

# # Train the model
# print(f"Training model for k={k_value}...")

# # Convert DataFrame to NumPy array (excluding CustomerID)
# features_np = features_for_training.values.astype('float32')

# # Create KMeans estimator
# kmeans = KMeans(role=role,
#                 instance_count=1,
#                 instance_type='ml.c4.xlarge',
#                 k=k_value,
#                 output_path=output_path)

# # Generate a random 3-digit serial number for the job name
# serial_number = generate_serial_number()
# # Append the serial number to the job name
# job_name = f"kmeans-{k_value}-{serial_number}"


# # Set the output path with job name
# model_output_path = f'{output_path}/{job_name}/output/'

# # Fit the model with the updated output path
# kmeans.fit(kmeans.record_set(features_np), job_name=job_name)

# print(f"Model for k={k_value} trained and saved to {model_output_path}")


## Model Validation

### Load Validation Data from S3

In [12]:
# validate_df = pd.read_csv('s3://team6datasets/splitDatasets/validation.csv')

In [13]:
# validate_df.shape

### Dimensionality Reduction using Variance 

In [14]:
# # Set the variance threshold
# variance_threshold = 0.2

# # Create a VarianceThreshold object
# selector = VarianceThreshold(threshold=variance_threshold)

# # Fit the selector to the data
# selector.fit(validate_df)

# # Get the support mask
# support = selector.get_support(indices=True)

# # Select features based on the support mask
# reduced_features_validate = validate_df.iloc[:, support]  

# # Now 'reduced_features' is a DataFrame containing features with variance exceeding the threshold
# print(f"Number of features removed: {len(validate_df.columns) - len(reduced_features_validate.columns)}")

In [15]:
# # Show the reduced_features_validate DataFrame
# print("Reduced Features DataFrame:")
# reduced_features_validate.head()

### Validate Model

Endpoint Deployment

In [16]:
# # Generate a random 3-digit serial number
# serial_number = ''.join(random.choices(string.digits, k=3))

# # Define the endpoint name with the serial number appended
# endpoint_name = f'kmeans-endpoint-{serial_number}'

# # Define the role
# role = get_execution_role()

# # Load the trained model
# model_key = f'kmeans_model_output/{job_name}/output/model.tar.gz'
# model_location = f's3://team6datasets/{model_key}'

# # Create a SageMaker Model object
# kmeans_model = sagemaker.KMeansModel(model_data=model_location, role=role)

# # Deploy the model to create an endpoint
# try:
#     predictor = kmeans_model.deploy(initial_instance_count=1,
#                                     instance_type='ml.m4.xlarge',
#                                     endpoint_name=endpoint_name)
#     print(f"Endpoint '{endpoint_name}' created successfully.")
# except Exception as e:
#     print(f"Failed to create endpoint '{endpoint_name}': {str(e)}")


In [17]:
# # Print IAM role for debugging
# role = get_execution_role()

# # Initialize S3 client
# s3 = boto3.client('s3')

# # Define the S3 location of the model
# model_key = f'kmeans_model_output/{job_name}/output/model.tar.gz'
# model_location = f's3://team6datasets/{model_key}'

# # Download the model
# s3.download_file('team6datasets', model_key, 'model.tar.gz')

# # Extract the model
# with tarfile.open('model.tar.gz', 'r:gz') as tar:
#     tar.extractall()
    
# # Path to the extracted model file
# model_path = './model_algo-1'

# # Load the model
# kmeans_model = KMeansModel(model_data=model_path, role=role)

# # Convert 'reduced_features_validate' DataFrame to NumPy array
# new_data_for_prediction = reduced_features_validate.drop(columns=['CustomerID'])
# new_data_np = new_data_for_prediction.values.astype('float32')

# # Create an instance of scikit-learn's KMeans and assign the underlying model
# kmeans_sklearn_model = KMeans()
# kmeans_sklearn_model.model = kmeans_model.cluster_model

# # Predict clusters for new data
# predictions = kmeans_sklearn_model.predict(new_data_np)


## Evaluating KMeans Model

In [18]:
# # Calculate silhouette score
# silhouette_avg = silhouette_score(new_data_np, predictions)
# print(f"Silhouette Score: {silhouette_avg}")



In [19]:
# inertia = kmeans_model.inertia_
# print(f"Inertia: {inertia}")

Predicting Clusters for Validation Data

In [20]:
# # Extract features for validation (excluding CustomerID)
# features_for_validation = reduced_features_validate.drop(columns=['CustomerID'])
# features_np_validation = features_for_validation.values.astype('float32')

# # Predict clusters for the validation data
# clusters = predictor.predict(features_np_validation)

# # Add the cluster labels to the validation DataFrame
# reduced_features_validate['Cluster'] = clusters

## Testing Model on Test Set

### Load Test Data Using S3

In [21]:
# test_df = pd.read_csv('s3://team6datasets/splitDatasets/test.csv')
# test_df.shape

### Dimensionality Reduction using Variance 

In [22]:
# # Set the variance threshold
# variance_threshold = 0.2

# # Create a VarianceThreshold object
# selector = VarianceThreshold(threshold=variance_threshold)

# # Fit the selector to the data
# selector.fit(test_df)

# # Get the support mask
# support = selector.get_support(indices=True)

# # Select features based on the support mask
# reduced_features_test = test_df.iloc[:, support]  

# # Now 'reduced_features' is a DataFrame containing features with variance exceeding the threshold
# print(f"Number of features removed: {len(test_df.columns) - len(reduced_features_test.columns)}")

### Predict Using Depoloyed Endpoint

In [23]:
# # Initialize the Predictor object with the endpoint name
# predictor = Predictor(endpoint_name=endpoint_name)

# # Convert the testing data to JSON format
# test_data = reduced_features_test.to_json(orient='records')

# # Perform inference
# predictions = predictor.predict(test_data)